## Audio experiments

In [224]:
#>flags -d:blas=cblas -p:~/code/misc/jnim

# Original Python code:
# srate = 11000
# dur = 1000
# rr = np.arange(dur*2)
# wave = np.sin(2*np.pi*400*rr/srate) * np.sin(0.5*np.pi*rr/dur)
# plt.plot(rr, wave)
# Audio(wave, rate=srate, autoplay=True)
    
import sequtils, std/[strutils, math, base64, strformat, times, os]
import arraymancer, pixie, jupyternimpkg/display
import wavfile, vorbis, fft

proc pixieHTML(img: Image): string =
    return fmt"""<img src="data:image/png;base64, {img.encodeImage(PngFormat).encode}" />"""

proc plot1D(img: Image, arr: openArray[float]): Image =
    var path: seq[string] = @[]
    
    let amax = arr[arr.maxIndex]
    let amin = arr[arr.minIndex]
    
    let mp = float(img.height)/(amax - amin)
    
    # For long arrays, draw <wmul> overlapping
    let wmul = 8
    if arr.len < img.width * wmul or false:
        for i in 0..<arr.len:
            let arr_v = arr[i]
            path.add(fmt"{int(float(i)*float(img.width)/float(arr.len))} {int((arr_v-amin) * mp)} ")
    else:
        for i in 0..<img.width * wmul:
            let arr_v = arr[int(float(i)*float(arr.len)/float(img.width)) div wmul]
            path.add(fmt"{i div wmul} {int((arr_v-amin) * mp)} ")
        
    img.strokePath(fmt"M {path[0]} L " & path.join(""), rgba(0, 0, 128, 255), strokeWidth = 1)
    return img

proc basicSine(srate: int): Tensor[system.float] =
    let srate = float(srate)
    let dur = 1000
    let rr_dur = dur * 2
    let rr = toSeq(0..<rr_dur).mapIt(it.float).toTensor()
    return sin(rr*(2.0*PI*440.0/srate)) *. sin(rr*(0.5*PI/float(dur)))


template benchmark(benchmarkName: string, code: untyped) =
  block:
    let t0 = epochTime()
    code
    let elapsed = epochTime() - t0
    let elapsedStr = elapsed.formatFloat(format = ffDecimal, precision = 3)
    echo "CPU Time [", benchmarkName, "] ", elapsedStr, "s"

let wsample = loadWav("sample.wav")
#html wsample.toHTML(false)
#html pixieHTML(newImage(500,100).plot1D(wsample.toFloat))

let srate = 11025
let wave = basicSine(srate).toSeq
#html Audio(wave, srate, false)
#html pixieHTML(newImage(400, 100).plot1D(wave))

#let vsample = loadVorbis("sample.ogg")
#html vsample.toHTML(false)
var vsf = wave.powerOf2Pad.mapIt(complex(it))
benchmark "fft":
    fft(vsf)
#ifft(vsf)
#let fftr = vsf.mapIt(it.re)
let fftr = vsf.mapIt(-abs(it))

#html Audio(fftr, wsample.freq, false)
#html pixieHTML(newImage(500,100).plot1D(fftr))
html pixieHTML(newImage(500,100).plot1D(fftr[0..fftr.len div 2]))

<img src="data:image/png;base64, iVBORw0KGgoAAAANSUhEUgAAAfQAAABkCAYAAABwx8J9AAAD/ElEQVR4Ae3cz+tlcxzH8de5N0mzUDaDhaJJykazEGXmnBtFpqGRJE3NaGTKr5KIwoz5C2zxF9hZ2N/OXZDFSBILC/oufLNQlLKwuBLbOUeT+r7ncx+P+uzf78/51HNx6y5T2oWLSX8sGTelxwSAA1Y86MMjSf4UdACYVjzo/Ykkfwg6AEwrHvTh8SS/CzoATCse9P6JJL8KOgBMKx704akkvwg6AEyrHvRnkvws6AAwrXrQTyf5SdABYFrxoPdnk+wJOgBMKx704bkkPwg6AEwrHvT+fJLvBR0AplUP+ktJvhN0AJhWPeivJPlG0AFgWvWgv5bkK0EHgGnFgz68keSyoAPAtOJB799K8oWgA8C04kEf3knymaDzPwlAq5a1F+svJtkk46b2nABwsAoH/cIiWVxKtmtBB4BphYM+XJ90bws6AMwrHPT7DiXXvSnoADCvcNAfujHpXhd0AJhXOOgP3JQsXxV0AJhXOOj94WTxsqADwLzCQT9+S7J4QdABYF7hoPe3JYvnBR0A5hUO+ur2pDsn6AAwr3DQjx9JFmcEHQDmFQ766q6kOy3oADCvcNCP350snhZ0AJhXOOj9PcniSUEHgHmFgz4cTbpTgg4A8yoH/d6ke0zQAa5K2C3Luuv29yfdo8l2nYybunMCwMErHPThWNI9LOgAMK9y0FdJ96CgA8C8wkHvh6RbCToAzKsc9F7QAeC/EXQAaICgA0ADBB0AGiDoANAAQQeABgg6ADRA0AGgAYIOAA0QdABogKADQAMEHQAasONBDwA0YVl3i75PulWyXSfjpu6cAHDwBB0AGiDoANAAQQeABgg6ADRA0AGgAYIOAA0QdABogKADQAMEHQAaIOgA0ABBB4AGCDoANEDQAaABgg4ADRB02BkBGrasu1vfJ90q2a6TcVN3TgA4eIIOAA0QdABogKADQAMEHQAaIOgA0ABBB4AGCDoANEDQAaABgg4ADRB0AGiAoANAAwQdABog6ADQAEEHgAYIOgANys5Z1t2475NulWzXybipOycAHDxBB4AGCDoANEDQAaABgg4ADRB0AGiAoANAAwQdABog6ADQAEEHgAYIOgA0QNABoAGVg34u2d6Q5Gv/5Q4A0woHfXg/6S4lOZuMH06vAQC7rWjQL9yR5Ezy3rNJfz4Zvk3Gvd3+VABwZUWDPpxItstk/CTpDyfdncm4vvIaALDblsm7HyT9yVonp5J8/s9v58OPyfajpD9Sa8Zqd3YV86Q/GccdeAPegDfw7xsYbk7689fmfYyfLpPh1qTbL3a+TLoxGfeT8bdk+DjpDhWbsdqdmafeO/ZNfBNv4Jp5Azma5MWk+7s3e9fO3N1+0u0n4+W/AOadU0xneeFxAAAAAElFTkSuQmCC" />

CPU Time [fft] 0.001s

